In [3]:
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav

metadata = pd.read_csv('./meta.csv')
metadata.head(10)

ModuleNotFoundError: No module named 'librosa'

<h1>#### Extracting MFCC's For every audio file</h1>

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [4]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-227.59056, 149.48346, -0.2679622, 6.281961, ...",crow
1,"[-217.32423, 147.93347, -2.3443086, 6.5044413,...",crow
2,"[-153.846, 124.30126, -29.816824, 8.058483, -3...",crow
3,"[-207.43484, 138.169, -14.45352, 18.09661, -7....",crow
4,"[-242.37227, 152.31898, 10.375188, 13.626253, ...",crow


In [5]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [6]:
X.shape

(620, 40)

In [7]:
# y

In [8]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [9]:
y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [10]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
X_train

array([[-3.64812469e+02,  6.22535706e+01, -7.71151495e+00, ...,
         1.85784250e-01,  1.94310904e+00,  2.23951077e+00],
       [-3.23632263e+02, -2.21011143e+01, -9.98628387e+01, ...,
         3.06606388e+00, -2.34944868e+00, -2.17550397e+00],
       [-3.12725372e+02,  2.83815689e+01, -9.98680191e+01, ...,
         1.35116637e+00,  1.59060371e+00,  7.19787300e-01],
       ...,
       [-2.14212769e+02,  1.05224625e+02, -2.28969803e+01, ...,
        -2.83538914e+00, -1.72062922e+00, -3.03701758e+00],
       [-2.01290176e+02,  1.28786163e+02,  2.21617222e+00, ...,
        -3.34015632e+00, -4.77180815e+00, -1.38094532e+00],
       [-3.79684570e+02,  7.36921844e+01, -6.73128815e+01, ...,
         2.80238152e+00,  2.78577876e+00,  1.82960975e+00]], dtype=float32)

In [12]:
y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [13]:
X_train.shape

(496, 40)

In [14]:
X_test.shape

(124, 40)

In [15]:
y_train.shape

(496, 3)

In [16]:
y_test.shape

(124, 3)

In [17]:
### model creation

In [18]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [20]:
### No of classes
num_labels=y.shape[1]

In [21]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [23]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [24]:
# Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
16/16 [==============================] - 3s 74ms/step - loss: 35.4931 - accuracy: 0.3920 - val_loss: 4.7686 - val_accuracy: 0.6210

Epoch 00001: val_loss improved from inf to 4.76858, saving model to saved_models/audio_classification.hdf5
Epoch 2/100
16/16 [==============================] - 0s 11ms/step - loss: 22.7736 - accuracy: 0.3830 - val_loss: 1.5835 - val_accuracy: 0.5484

Epoch 00002: val_loss improved from 4.76858 to 1.58352, saving model to saved_models/audio_classification.hdf5
Epoch 3/100
16/16 [==============================] - 0s 10ms/step - loss: 14.7430 - accuracy: 0.4134 - val_loss: 1.3698 - val_accuracy: 0.5484

Epoch 00003: val_loss improved from 1.58352 to 1.36977, saving model to saved_models/audio_classification.hdf5
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 11.0606 - accuracy: 0.3745 - val_loss: 0.9062 - val_accuracy: 0.5081

Epoch 00004: val_loss improved from 1.36977 to 0.90622, saving model to saved_models/audio_class

16/16 [==============================] - 0s 5ms/step - loss: 0.6803 - accuracy: 0.6766 - val_loss: 0.6010 - val_accuracy: 0.7177

Epoch 00074: val_loss improved from 0.61982 to 0.60097, saving model to saved_models/audio_classification.hdf5
Epoch 75/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7107 - accuracy: 0.6576 - val_loss: 0.5895 - val_accuracy: 0.7177

Epoch 00075: val_loss improved from 0.60097 to 0.58952, saving model to saved_models/audio_classification.hdf5
Epoch 76/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6895 - accuracy: 0.6933 - val_loss: 0.5892 - val_accuracy: 0.7258

Epoch 00076: val_loss improved from 0.58952 to 0.58923, saving model to saved_models/audio_classification.hdf5
Epoch 77/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6553 - accuracy: 0.7025 - val_loss: 0.5836 - val_accuracy: 0.7258

Epoch 00077: val_loss improved from 0.58923 to 0.58360, saving model to saved_models/audio_classification.hd

In [25]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8709677457809448


In [26]:
# prediction_feature.shape

In [27]:
X_test[1]

array([-2.72459106e+02,  1.02502556e+02, -4.11994438e+01,  4.55670452e+00,
       -1.65558758e+01,  2.48143063e+01, -8.10355783e-01,  1.79815884e+01,
       -7.88385582e+00,  1.79162884e+01, -4.71778584e+00,  1.36451921e+01,
       -3.56350684e+00,  1.02704239e+01,  1.57364357e+00,  9.26171970e+00,
       -3.06813791e-02,  8.73383999e+00,  3.96176791e+00,  6.96624327e+00,
        1.93785226e+00,  6.31154442e+00,  2.83724236e+00,  3.94089293e+00,
        3.68973994e+00,  5.17122507e+00,  2.06391811e+00,  3.26116681e+00,
        1.60792029e+00,  5.27418947e+00,  2.13462853e+00,  2.71737528e+00,
        4.05507135e+00,  2.00532413e+00,  3.22364306e+00,  4.26191378e+00,
        2.80193353e+00,  3.44579190e-01,  3.78335667e+00,  1.30365098e+00],
      dtype=float32)

In [28]:
model.predict_classes(X_test)

/home/maulik/anaconda3/envs/demo/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1,
       0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0,
       2, 2, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1])

<h1>model save</h1>

In [29]:
# model.save('audio_classification_model')

In [30]:
##Testing Some Test Audio Data¶
# Steps

# Preprocess the new audio data
# predict the classes
# Invere transform your Predicted Label

In [31]:
# loaded_model = tf.keras.models.load_model('audio_classification_model')

In [36]:
filename="/home/maulik/Desktop/Maulik_Gupta/Documents/jatayu_openunmix/open-unmix-pytorch/downloaded_audio/bbird_crowacc.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

# print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
# print(mfccs_scaled_features)
# print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
# print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

array(['blackbird'], dtype='<U9')